We will use this dataset that contains Customer Id, Surname, Credit Score, Geography, Gender, Age, Tenure, Balance, Num of Products they use from the bank such as credit card or loan, etc, Has Credit card or not (1 means yes 0 means no), Is Active Member (the customer is using the bank or not), and estimated salary. The last column is the dependent variable - has customer exited or not from the bank in the future (1 means the customer has exited the bank and 0 means the customer has stayed with the bank). The bank wants to use this data to see if they can predict if a new customer will stay with the bank or not. Build a classifier using neural networks. Make appropriate design decisions for its architecture.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')

# Extract predictor variables and outcome variable
cols = dataset.columns[3:13]
X = pd.DataFrame(dataset.iloc[:, 3:13].values)
y = dataset.iloc[:, 13].values

# Turn category labels for gender to 0 and 1

labelencoder_X_2 = LabelEncoder()
X.loc[:, 2] = labelencoder_X_2.fit_transform(X.iloc[:, 2])

# Change GEography categorical variable into 0, 1, and 2 
labelencoder_X_1 = LabelEncoder()
X.loc[:, 1] = labelencoder_X_1.fit_transform(X.iloc[:, 1])

In [3]:
# Now do one hot encoding
#onehotencoder = OneHotEncoder(categorical_features = [1])
#X = onehotencoder.fit_transform(X).toarray()
#X = X[:, 1:]

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(columnTransformer.fit_transform(X), dtype = np.float64)
X = X[:, 1:]

In [4]:
# Create train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Scale the data -- one reason is to save on computing later
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [5]:
# Sequential() allows building an NN with a sequence of layers
nn = Sequential()
nn.add(Dense(11, activation = 'relu'))
nn.add(Dense(7, activation = 'relu'))
nn.add(Dense(1, activation = 'sigmoid'))

# 'adam' for stochastic gradient descent
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [6]:
nn.fit(X_train, y_train)
nn.summary()

250/250 [==============================] - 0s 534us/step - loss: 0.6420 - accuracy: 0.6219
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (32, 11)                  132       
_________________________________________________________________
dense_1 (Dense)              (32, 7)                   84        
_________________________________________________________________
dense_2 (Dense)              (32, 1)                   8         
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


In [7]:
y_pred = nn.predict(X_test)
y_pred = (y_pred > 0.5)

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8005
[[1598    0]
 [ 399    3]]
              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1598
           1       1.00      0.01      0.01       402

    accuracy                           0.80      2000
   macro avg       0.90      0.50      0.45      2000
weighted avg       0.84      0.80      0.71      2000

